<a href="https://colab.research.google.com/github/shu65/plamo-2-8b-lora-sft-example/blob/main/PLaMo_2_8B_LoRA_SFT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch==2.4.1 torchvision==0.19.1 torchaudio==2.4.1 --index-url https://download.pytorch.org/whl/cu124

Looking in indexes: https://download.pytorch.org/whl/cu124
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.2/797.2 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 111.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 89.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 89.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.4/883.4 kB 54.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 115.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.4/128.4 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install trl numba>=0.60.0 mamba-ssm>=2.2.2 causal-conv1d>=1.4.0 transformers>=4.44.2

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.


In [ ]:
!pip list | grep -e torch -e transformers -e numba -e mamba-ssm -e causal-conv1d

causal-conv1d                      1.5.0.post8
mamba-ssm                          2.2.4
numba                              0.60.0
numba-cuda                         0.2.0
sentence-transformers              3.4.1
torch                              2.4.1+cu124
torchaudio                         2.4.1+cu124
torchsummary                       1.5.1
torchvision                        0.19.1+cu124
transformers                       4.50.0


In [ ]:
from huggingface_hub import login

login()

In [ ]:
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer
import datasets
import string
from trl import DataCollatorForCompletionOnlyLM
from trl import SFTConfig, SFTTrainer
from peft import LoraConfig
import torch

In [ ]:
model_name = "pfnet/plamo-2-8b"
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True, torch_dtype=torch.bfloat16).to("cuda")
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

modeling_plamo.py:   0%|          | 0.00/66.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/pfnet/plamo-2-8b:
- modeling_plamo.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
You are using a model of type plamo2 to instantiate a model of type plamo. This is not supported for all configurations of models and can yield errors.


model.safetensors.index.json:   0%|          | 0.00/38.9k [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenization_plamo.py:   0%|          | 0.00/16.9k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/pfnet/plamo-2-8b:
- tokenization_plamo.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


tokenizer.jsonl:   0%|          | 0.00/3.32M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/587 [00:00<?, ?B/s]

In [ ]:
dataset = datasets.load_dataset("kunishou/databricks-dolly-15k-ja")
train_dataset = dataset["train"].filter(lambda data: "instruction" in data and "output" in data and data["input"] == "").select(range(2000))

print("Train dataset")
print(train_dataset)
print(train_dataset[0])

README.md:   0%|          | 0.00/499 [00:00<?, ?B/s]

databricks-dolly-15k-ja.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/15015 [00:00<?, ? examples/s]

Filter:   0%|          | 0/15015 [00:00<?, ? examples/s]

Train dataset
Dataset({
    features: ['output', 'input', 'index', 'category', 'instruction'],
    num_rows: 2000
})
{'output': 'イコクエイラクブカ', 'input': '', 'index': '1', 'category': 'classification', 'instruction': '魚の種類はどっち？イコクエイラクブカとロープ'}


In [ ]:
data_collator = DataCollatorForCompletionOnlyLM(
    response_template=tokenizer.encode(" Answer:\n", add_special_tokens=False),
    tokenizer=tokenizer
)

In [ ]:
peft_config = LoraConfig(
    task_type="CAUSAL_LM",
    target_modules=[
        "qkv_proj",
    ],
)

In [ ]:
sft_args = SFTConfig(
    output_dir="./outputs",
    evaluation_strategy="no",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=5e-5,
    num_train_epochs=1.0,
    lr_scheduler_type="cosine",
    warmup_ratio=0.3,
    logging_steps=10,
    save_strategy="epoch",
    report_to="tensorboard",
    bf16=True,
    max_seq_length=1024,
    gradient_checkpointing=True,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
INSTRUCTION_TEMPLATE = string.Template(
    """### Question:
${input}

### Answer:
${response}<|plamo:eos|>
"""
)

def formatting_func(example):
  text = INSTRUCTION_TEMPLATE.substitute(input=example["instruction"], response=example["output"])
  return text

In [ ]:
trainer = SFTTrainer(
    model=model,
    args=sft_args,
    peft_config=peft_config,
    data_collator=data_collator,
    train_dataset=train_dataset,
    formatting_func=formatting_func,
)

trainer.train()
trainer.save_model()

The repository for pfnet/plamo-2-8b contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/pfnet/plamo-2-8b.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Applying formatting function to train dataset:   0%|          | 0/2000 [00:00<?, ? examples/s]

Converting train dataset to ChatML:   0%|          | 0/2000 [00:00<?, ? examples/s]

Parameter 'fn_kwargs'={'tokenizer': PlamoTokenizer(name_or_path='pfnet/plamo-2-8b', vocab_size=100000, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|plamo:bos|>', 'eos_token': '<|plamo:eos|>', 'unk_token': '<|plamo:unk|>', 'pad_token': '<|plamo:pad|>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("<|plamo:unk|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<|plamo:bos|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("<|plamo:eos|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<|plamo:pad|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)} of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your 

Applying chat template to train dataset:   0%|          | 0/2000 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/2000 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/2000 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss
10,3.963900
20,4.248200
30,4.668600
40,3.354200
50,3.871100
60,3.698500
70,4.551100
80,4.426700
90,3.377000
100,4.067300


/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
You are using a model of type plamo2 to instantiate a model of type plamo. This is not supported for all configurations of models and can yield errors.
You are using a model of type plamo2 to instantiate a model of type plamo. This is not supported for all configurations of models and can yield errors.


In [ ]:
import torch

# プロンプトの準備
prompt = "### Question:\n埼玉の県庁所在地は何市？\n\n### Answer:\n"

# 推論の実行
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
generated_tokens = trainer.model.generate(
    **inputs,
    max_new_tokens=64,
    pad_token_id=tokenizer.pad_token_id,
)[0]
generated_text = tokenizer.decode(generated_tokens)
print(generated_text)

/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:92: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


<|plamo:bos|>### Question:
埼玉の県庁所在地は何市？

### Answer:
さいたま市<|plamo:eos|>
